# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298416645800                   -0.85    5.0         
  2   -8.300193854622       -2.75       -1.25    1.0    114ms
  3   -8.300430225088       -3.63       -1.89    2.2    160ms
  4   -8.300461941200       -4.50       -2.77    3.0    132ms
  5   -8.300464135539       -5.66       -3.07    3.0    146ms
  6   -8.300464368117       -6.63       -3.23    4.2    145ms
  7   -8.300464499183       -6.88       -3.37    6.0    197ms
  8   -8.300464570736       -7.15       -3.51    2.4    164ms
  9   -8.300464625587       -7.26       -3.73    1.4    157ms
 10   -8.300464628832       -8.49       -3.78    1.0    100ms
 11   -8.300464640304       -7.94       -4.00    1.9    135ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64213490072                   -0.70    6.9         
  2   -16.64603160853       -2.41       -1.14    1.0    242ms
  3   -16.67920263699       -1.48       -1.87    2.2    271ms
  4   -16.67927883900       -4.12       -2.67    5.1    403ms
  5   -16.67928595583       -5.15       -2.99    4.4    386ms
  6   -16.67928619162       -6.63       -3.31    1.9    270ms
  7   -16.67928621621       -7.61       -3.76    1.9    257ms
  8   -16.67928622118       -8.30       -4.52    2.0    291ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32476727370                   -0.56    6.9         
  2   -33.33260340434       -2.11       -1.00    1.0    997ms
  3   -33.33408456364       -2.83       -1.74    3.2    1.19s
  4   -33.33426136460       -3.75       -2.64    3.9    1.12s
  5   -33.33551540550       -2.90       -2.49    5.8    1.52s
  6   -33.33694281576       -2.85       -2.49    7.8    1.31s
  7   -33.33694257309   +   -6.61       -2.49    1.1    912ms
  8   -33.33691860703   +   -4.62       -2.47    2.2    980ms
  9   -33.33544391192   +   -2.83       -1.97    3.4    1.47s
 10   -33.33493848733   +   -3.30       -1.92    1.6    970ms
 11   -33.33437086778   +   -3.25       -1.87    1.5    991ms
 12   -33.33510910887       -3.13      

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298460903190                   -0.85    5.2         
  2   -8.300274264027       -2.74       -1.59    1.1   88.9ms
  3   -8.300435879713       -3.79       -2.64    3.2    123ms
  4   -8.300420846338   +   -4.82       -2.43    7.0    193ms
  5   -8.300464411066       -4.36       -3.39    1.0   89.7ms
  6   -8.300464616816       -6.69       -3.94    3.4    158ms
  7   -8.300464641867       -7.60       -4.36    1.6    108ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32753450080                   -0.56    7.0         
  2   -33.31878819076   +   -2.06       -1.27    1.4    826ms
  3   -16.68140120307   +    1.22       -0.45    6.4    1.73s
  4   -33.32656845068        1.22       -1.90    5.9    1.56s
  5   -33.28254828963   +   -1.36       -1.42    3.1    1.38s
  6   -33.26758229320   +   -1.82       -1.62    3.5    1.30s
  7   -33.12802083828   +   -0.86       -1.40    5.0    1.39s
  8   -33.33630273999       -0.68       -2.53    4.1    1.26s
  9   -33.33562128123   +   -3.17       -2.42    3.1    1.29s
 10   -33.33680881767       -2.93       -2.70    2.4    1.05s
 11   -33.33684869493       -4.40       -3.00    1.8    919ms
 12   -33.33694006477       -4.04      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.